In [1]:
#import nessesary library for data pre-processing
from keras.preprocessing.image import ImageDataGenerator, img_to_array, array_to_img

#Use ImageDataGenerator for data augmentation to get more data for training and rescaling validation and testing data
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=50, zoom_range=0.4, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('train', target_size=(244,244), batch_size=10, class_mode='binary')
valid_generator = test_datagen.flow_from_directory('valid', target_size=(244,244), batch_size=10, class_mode='binary')
test_generator = test_datagen.flow_from_directory('test', target_size=(244,244), batch_size=10, class_mode='binary')

Using TensorFlow backend.


Found 161 images belonging to 2 classes.
Found 54 images belonging to 2 classes.
Found 54 images belonging to 2 classes.


In [2]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(16, (3, 3), input_shape=(244, 244, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
 
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 242, 242, 16)      448       
_________________________________________________________________
activation_1 (Activation)    (None, 242, 242, 16)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 121, 121, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 119, 119, 32)      4640      
_________________________________________________________________
activation_2 (Activation)    (None, 119, 119, 32)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 59, 59, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 57, 57, 64)        18496     
__________

In [3]:
from keras.callbacks import ModelCheckpoint

#use ModelCheckpoint to save weights with the best accuracy
checkpointer = ModelCheckpoint(filepath='weights.best.h5py', monitor='val_acc', verbose=1, save_best_only=True, mode='max')

#train the network on train_generator dataset
model.fit_generator(train_generator, steps_per_epoch=145, epochs=7, validation_data=valid_generator, validation_steps=62,\
                    callbacks=[checkpointer], verbose=1)


Epoch 1/7
145/145 [==============================] - 245s 2s/step - loss: 0.4958 - acc: 0.7910 - val_loss: 0.3379 - val_acc: 0.8148

Epoch 00001: val_acc improved from -inf to 0.81481, saving model to weights.best.h5py
Epoch 2/7
145/145 [==============================] - 245s 2s/step - loss: 0.2950 - acc: 0.8759 - val_loss: 0.3342 - val_acc: 0.9074

Epoch 00002: val_acc improved from 0.81481 to 0.90741, saving model to weights.best.h5py
Epoch 3/7
145/145 [==============================] - 243s 2s/step - loss: 0.2095 - acc: 0.9145 - val_loss: 0.4951 - val_acc: 0.8148

Epoch 00003: val_acc did not improve from 0.90741
Epoch 4/7
145/145 [==============================] - 242s 2s/step - loss: 0.2016 - acc: 0.9352 - val_loss: 0.4686 - val_acc: 0.8333

Epoch 00004: val_acc did not improve from 0.90741
Epoch 5/7
145/145 [==============================] - 242s 2s/step - loss: 0.1505 - acc: 0.9476 - val_loss: 0.2106 - val_acc: 0.9630

Epoch 00005: val_acc improved from 0.90741 to 0.96296, savin

In [4]:
def accuracy(path):
    
    #load the weights with the best accuracy value
    model.load_weights('weights.best.h5py')
    
    test_data = test_datagen.flow_from_directory(path, target_size=(244,244), batch_size=10, class_mode='binary')
    
    #Evaluate the model on the test set
    scores = model.evaluate_generator(test_generator,42)
    return  scores[1]

In [5]:
#here as the argument to the accuracy use the path to your test data
print ("Acuraccy: ", accuracy('test'))

Found 54 images belonging to 2 classes.
Acuraccy:  0.8333333248183841
